# Falicov-Kimball Model

In [ ]:
using KadanoffBaym
using LinearAlgebra
using LsqFit
using JLD
using UnPack

using PyPlot
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")

font_size = 16
# font_size = 20

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=font_size)
PyPlot.matplotlib.rc("axes", labelsize=font_size)
PyPlot.matplotlib.rc("xtick.major", size=8)
PyPlot.matplotlib.rc("ytick.major", size=8)
PyPlot.matplotlib.rc("xtick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("ytick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("xtick", top=true, direction="inout")
PyPlot.matplotlib.rc("ytick", right=true, direction="inout")

## Model

### Non-Hermitian Hamiltonian:

$$
\begin{align}\begin{split}
    \hat{H} &= \varepsilon \left( f^{\dagger}_1 f^\phantom{\dagger}_1  +  f^{\dagger}_2 f^\phantom{\dagger}_2 \right) - J \left(c^{\dagger}_1 c^\phantom{\dagger}_2 + c^{\dagger}_2 c^\phantom{\dagger}_1\right) + U \left( f^{\dagger}_1 f^\phantom{\dagger}_1 c^{\dagger}_1 c^\phantom{\dagger}_1  +  f^{\dagger}_2 f^\phantom{\dagger}_2 c^{\dagger}_2 c^\phantom{\dagger}_2 \right)
\end{split}\end{align}
$$

$$
    G^>_{ij}(t, t') = -i \langle c^\phantom{\dagger}_i(t) c^{\dagger}_j(t') \rangle\\
    F^>_{ij}(t, t') = -i \langle f^\phantom{\dagger}_i(t) f^{\dagger}_j(t') \rangle
$$

$$
    \Sigma^{\mathrm{HF}}_{c,\,ij}(t, t') = {\mathrm{i}}\delta_{ij}\delta(t - t') F^<_{ii}(t, t)\\
    \Sigma^{\mathrm{HF}}_{f,\,ij}(t, t') = {\mathrm{i}}\delta_{ij}\delta(t - t') G^<_{ii}(t, t)
$$

Not yet any guarantee for the prefactors!
$$
    \Sigma^{\mathrm{NCA}}_{c,\,ij}(t, t') = {\mathrm{i}\cdot (-\mathrm{i}) \cdot 2 U^2} G_{ij}(t, t') F_{ij}(t, t') F_{ji}(t', t)\\
    \Sigma^{\mathrm{NCA}}_{f,\,ij}(t, t') = {\mathrm{i}\cdot (-\mathrm{i}) \cdot 2 U^2} F_{ij}(t, t') G_{ij}(t, t') G_{ji}(t', t)
$$

### Equations of motion

#### Vertical Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_t - \varepsilon_1  & -J \\
    -J & i \partial_t - \varepsilon_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t')  \\
    0 &= \begin{pmatrix}
    i \partial_t - \varepsilon_1  & -J \\
    -J & i \partial_t - \varepsilon_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Horizontal Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \varepsilon_1  & J \\
    J & i \partial_{t'} + \varepsilon_2
    \end{pmatrix}  \\
    0 &= \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \varepsilon_1  & J \\
    J & i \partial_{t'} + \varepsilon_2
    \end{pmatrix}  
\end{split}\end{align}

#### Equal-Time.

\begin{align}\begin{split}
    0 &= i \partial_T
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)
    - \left[\begin{pmatrix}
    \varepsilon_1 & J \\
    J & \varepsilon_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)\right]    \\
    0 &= i \partial_T
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)  
    - \left[\begin{pmatrix}
    \varepsilon_1 & J \\
    J & \varepsilon_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)\right]
\end{split}\end{align}

## Solving

In [ ]:
function integrate(x::AbstractVector, y::AbstractVector)
    if isone(length(x))
        return zero(first(y))
    end

    @inbounds retval = (x[2] - x[1]) * (y[1] + y[2])
    @inbounds @fastmath @simd for i in 2:(length(y) - 1)
        retval += (x[i+1] - x[i]) * (y[i] + y[i+1])
    end
    return 1//2 * retval
end;

In [ ]:
# Self-energy
function self_energies!(nca, t, t′)
    @unpack GL, GG, FL, FG, ΣNCA_c_L, ΣNCA_c_G, ΣNCA_f_L, ΣNCA_f_G, U = nca
    
    if (n = size(GL, 3)) > size(ΣNCA_c_L, 3)
        resize!(ΣNCA_c_L, n)
        resize!(ΣNCA_c_G, n)
        resize!(ΣNCA_f_L, n)
        resize!(ΣNCA_f_G, n)        
    end
        
    ΣNCA_c_L[t, t′] = 1.0im .* (-1.0im .* U^2) .* GL[t, t′] .* FL[t, t′] .* transpose(FG[t′, t])
    ΣNCA_c_G[t, t′] = 1.0im .* (-1.0im .* U^2) .* GG[t, t′] .* FG[t, t′] .* transpose(FL[t′, t])
    
    ΣNCA_f_L[t, t′] = 1.0im .* (-1.0im .* U^2) .* FL[t, t′] .* GL[t, t′] .* transpose(GG[t′, t])
    ΣNCA_f_G[t, t′] = 1.0im .* (-1.0im .* U^2) .* FG[t, t′] .* GG[t, t′] .* transpose(GL[t′, t])
    
#     println(ΣNCA_f_L[t, t′])
end

# Container for non-crossing approximation
struct NCA
    GL::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    GG::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    FL::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    FG::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    ΣNCA_c_L::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    ΣNCA_c_G::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    ΣNCA_f_L::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    ΣNCA_f_G::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    
    U::Float64
  
    # Initialize problem
    function NCA(GL0::Matrix{ComplexF64}, FL0::Matrix{ComplexF64}, U::Float64)
        nca = new(
          GreenFunction(reshape(GL0, size(GL0)..., 1, 1), SkewHermitian),
          GreenFunction(reshape(GL0 - 1.0im * I, size(GL0)..., 1, 1), SkewHermitian),
          GreenFunction(reshape(FL0, size(FL0)..., 1, 1), SkewHermitian),
          GreenFunction(reshape(FL0 - 1.0im * I, size(FL0)..., 1, 1), SkewHermitian),            
          GreenFunction(zeros(ComplexF64, size(GL0)..., 1, 1), SkewHermitian),
          GreenFunction(zeros(ComplexF64, size(GL0)..., 1, 1), SkewHermitian),
          GreenFunction(zeros(ComplexF64, size(FL0)..., 1, 1), SkewHermitian),
          GreenFunction(zeros(ComplexF64, size(FL0)..., 1, 1), SkewHermitian),            
          U
        )

        # Initialize self-energies
        self_energies!(nca, 1, 1)

        return nca
    end
end

In [ ]:
# final time
T = 10.0

# Hamiltonian
ε = 1.0
J = 1.0
H = ComplexF64[0 -J 0 0; -J 0 0 0; 0 0 ε 0; 0 0 0 ε]
U = 5.0;

In [ ]:
t_scale = J;

In [ ]:
# quantum numbers
dim = 2

# Define your Green functions at (t0, t0), time-arguments at the end!
GL0 = zeros(ComplexF64, dim, dim)
FL0 = zeros(ComplexF64, dim, dim)

# initial condition
N_c = [0.5, 0.1]
N_f = [0.3, 0.7]

GL0[1, 1] = 1.0im * N_c[1]
GL0[2, 2] = 1.0im * N_c[2]
FL0[1, 1] = 1.0im * N_f[1]
FL0[2, 2] = 1.0im * N_f[2]

# NCA data
nca = NCA(GL0, FL0, U);

u0 = [nca.GL, nca.GG, nca.FL, nca.FG]

function f_vert(nca, u, times, t, t′)
    @unpack GL, GG, FL, FG, ΣNCA_c_L, ΣNCA_c_G, ΣNCA_f_L, ΣNCA_f_G, U = nca
    
    # real-time collision integral
    ∫dt(i, j, A, B) = sign(j-i) * integrate(times[min(i, j):max(i, j)], 
                                          [A[t, s] .* B[s, t′] for s=min(i, j):max(i, j)])    
        
    # normal operator ordering
    ΣHF_c(t, t′) = 1.0im * U * [FL[1, 1, t, t] 0; 0 FL[2, 2, t, t]]
    ΣHF_f(t, t′) = 1.0im * U * [GL[1, 1, t, t] 0; 0 GL[2, 2, t, t]]
        
    f_GL = (- 1.0im * (H[1:2, 1:2] + ΣHF_c(t, t′)) * GL[t, t′] 
            - 1.0im * (∫dt(1, t, ΣNCA_c_G, GL) - ∫dt(1, t, ΣNCA_c_L, GL) + ∫dt(1, t′, ΣNCA_c_L, GL) - ∫dt(1, t′, ΣNCA_c_L, GG)))
    
    f_GG = (- 1.0im * (H[1:2, 1:2] + ΣHF_c(t, t′)) * GG[t, t′]
            - 1.0im * (∫dt(1, t, ΣNCA_c_G, GG) - ∫dt(1, t, ΣNCA_c_L, GG) + ∫dt(1, t′, ΣNCA_c_G, GL) - ∫dt(1, t′, ΣNCA_c_G, GG)))
    
    f_FL = (- 1.0im * (H[3:4, 3:4] + ΣHF_f(t, t′)) * FL[t, t′]
            - 1.0im * (∫dt(1, t, ΣNCA_f_G, FL) - ∫dt(1, t, ΣNCA_f_L, FL) + ∫dt(1, t′, ΣNCA_f_L, FL) - ∫dt(1, t′, ΣNCA_f_L, FG)))
    
    f_FG = (- 1.0im * (H[3:4, 3:4] + ΣHF_f(t, t′)) * FG[t, t′]
            - 1.0im * (∫dt(1, t, ΣNCA_f_G, FG) - ∫dt(1, t, ΣNCA_f_L, FG) + ∫dt(1, t′, ΣNCA_f_G, FL) - ∫dt(1, t′, ΣNCA_f_G, FG)))   

    
    return [f_GL, f_GG, f_FL, f_FG]
end

# f_diag = (u, times, t) -> f_vert(nca, u, times, t, t) - adjoint.(f_vert(nca, u, times, t, t));

function f_diag(u, times, t)
    println(" t: $(times[t])")
    result = f_vert(nca, u, times, t, t)
    return  result - adjoint.(result)
end;

In [ ]:
# Basically this will mutate Lesser and Greater in place
rtol = 1e-5
atol = 1e-2rtol
sol = kbsolve((u, args...) -> f_vert(nca, u, args...), f_diag, u0, (0.0, T); 
              callback = (_, args...) -> self_energies!(nca, args...), atol=atol, rtol=rtol);

In [ ]:
# stop = Int(n/2) + 1 

times = sol.t # range(0, length=n + 1, stop=T) |> collect;

n = length(sol.t) - 1

horizontal_times = k -> vcat(times[k:end] .- times[k], times[end] .+ (1:(k - 1) |> collect) .* (times[end] - times[end-1]));

In [ ]:
n

## Example plots

In [ ]:
T = sol.t[end]
t_scale = (J == 0. ? 1. : J)
ω_scale = (J == 0. ? 1. : 1/J);

In [ ]:
GL, FL, GG, FG = nca.GL, nca.FL, nca.GG, nca.FG;

In [ ]:
idx1 = 1
idx2 = 1;

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
plot(J .* sol.t, [imag(GL.data[1, 1, k, k]) for k in 1:length(sol.t)], label="\$ i=1\$", ls="-", c="C0")
plot(J .* sol.t, [imag(GL.data[2, 2, k, k]) for k in 1:length(sol.t)], label="\$ i=2\$", ls="--", c="C0")
xlim(0, T)
ylim(0, 0.5)
xticks(J .* [2t for t in 0:T/2])
yticks([0, 0.25 ,0.5, 0.75])
xlabel("\$J t\$")
ylabel("\$\\mathrm{Im}\\; G^<_{ii}(t, t)\$")
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_axisbelow(false)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))
ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)

ax = subplot(122)
plot(J .* sol.t, [(sum(GL.data[i, i, k, k] for i in 1:2) |> imag) for k in 1:length(sol.t)] .- (N_c[1] + N_c[2]) .|> abs, label="\$ c \$", ls="--", c="C0")
plot(J .* sol.t, [(sum(FL.data[i, i, k, k] for i in 1:2) |> imag) for k in 1:length(sol.t)] .- (N_f[1] + N_f[2]) .|> abs, label="\$ f \$", ls="--", c="C1")
xlim(0, T)
xticks(J .* [2t for t in 0:T/2])
yticks([0., 0.5e-15, 1e-15])
xlabel("\$J t\$")
# ylabel("\$\\mathrm{Tr} \\left|G^<(t, t) - G^<(0, 0) \\right|\$", labelpad=16)
ylabel("Number conservation", labelpad=16)
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.yaxis.set_label_position("right")
# ax.set_axisbelow(false)
# ax.yaxis.tick_right()
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))
ax.legend(loc="upper left", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)

tight_layout(pad=0.1, w_pad=0.5, h_pad=0)
# savefig("falicov_kimball_example_T.pdf")

In [ ]:
using FFTW, Interpolations

In [ ]:
idx = 1;
new_times = range(first(sol.t), stop=last(sol.t), length=2049);

In [ ]:
ρ_kb = interpolate((sol.t, sol.t), view(FG.data .- FL.data, idx, idx, :, : ), Gridded(Linear()));

In [ ]:
ρ_kb_wigner, (taus, ts) = wigner_transform([ρ_kb(t1, t2) for t1 in new_times, t2 in new_times]; ts=new_times, fourier=false);
ρ_kb_FFT, (ωs, ts) = wigner_transform([ρ_kb(t1, t2) for t1 in new_times, t2 in new_times]; ts=new_times, fourier=true);

In [ ]:
xpad = 8
ypad = 5

center = floor(length(new_times) / 2) |> Int

figure(figsize=(7, 3))

ax = subplot(121)
plot(t_scale * taus, -ρ_kb_wigner[:, center] |> imag, ls="-", c="C0", lw=1.5)
ax.set_xlabel("\$J \\tau\$")
ax.set_xlim(-t_scale * T, t_scale * T)
ax.set_ylim(0, 1.0)
ax.set_xticks(t_scale .* [-T, -T/2, 0, T/2, T])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel("\$ \\left| A^F_{11}(\\tau, T)\\right|\$")

plot_kb = ρ_kb_FFT[:, center] |> real

ax = subplot(122)
plot(ω_scale * ωs, plot_kb, "-", c="C0", lw=1.5)

ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_xlabel("\$\\omega/J\$")
ax.set_ylabel("\$ A^F_{11}(\\omega, T)\$", labelpad=16)
ax.set_xlim(10 .* (-1, 1))
ax.set_ylim(0, )
ax.yaxis.set_label_position("right")
# ax.legend(loc="best", handlelength=1.4, frameon=false, borderpad=0, labelspacing=0.25)

tight_layout(pad=0.1, w_pad=0.5, h_pad=0)


# savefig("falicov_kimball_example_tau_omega.pdf")

In [ ]:
Y, X = np.meshgrid(sol.t, sol.t);

In [ ]:
# X_gauss = X
# Y_gauss = Y
# FL_gauss = FL
# FG_gauss = FG;

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121) # plt.gca()
heatmap = ax.pcolormesh(t_scale * X_gauss, Y_gauss, imag(FL_gauss.data[1, 1, :, :]) .- imag(FG_gauss.data[1, 1, :, :]), cmap=cmap, rasterized=true) # vmin=vmin, vmax=vmax,
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$J t\$")
ax.set_ylabel("\$J t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])

ax = subplot(122)
heatmap = ax.pcolormesh(t_scale * X, Y, imag(FL.data[1, 1, :, :]) .- imag(FG.data[1, 1, :, :]), cmap=cmap, rasterized=true) # vmin=vmin, vmax=vmax,
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$J t\$")
# ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])
ax.set_yticklabels([])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)

# savefig("falicov_kimball_example_two_times.pdf")

## Testing

In [ ]:
cmap = "gist_heat";

In [ ]:
figure(figsize=(5, 3))
ax = plt.gca()
heatmap = ax.pcolormesh(t_scale * X, Y, imag(GL.data[1, 1, :, :]) .- imag(GG.data[1, 1, :, :]), cmap=cmap, rasterized=true) # vmin=vmin, vmax=vmax,
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
# ax.set_xlabel("\$\\lambda T\$")
# ax.set_ylabel("\$\\omega\$")
# ax.set_xlim(0, t_scale * T)
# ax.set_xticks([0, t_scale * T/2, t_scale * T])
# ax.set_yticks([-20, -10, 0, 10, 20])
# ax.set_ylim(-20, 20)
colorbar(mappable=heatmap)
tight_layout(pad=0.0, w_pad=0, h_pad=0)